In [12]:
import pandas as pd
import numpy as np
import os
from bokeh.plotting import figure
from bokeh.io import output_file, show, save
from functies_voor_selectief_plotten import *
from sklearn.metrics import mean_squared_error
from bokeh.models import HoverTool, BoxSelectTool
from products_per_country import *
from bokeh.plotting import figure, output_file, show, ColumnDataSource

In [14]:
# controleer altijd of je in de map helpfiles zit
# os.chdir('../')
# os.chdir('helpfiles')
print(os.getcwd())

C:\Users\boble\Documents\GitHub\DataProcessing\datasets


In [13]:
from select_df import select_df

df_foodprices = select_df(1, 'foodprices')
df_migration = select_df(1, 'migration_movements')
df_weather = select_df(1, 'temperature_and_precipitation')

Succesfully imported foodprices!
Succesfully imported migration_movements!
succesfully imported temperature_and_precipitation!


In [52]:
# plot genormaliseerde foodprices van een product tegenover migration movements in een bepaald land
# 95% van de scatters ligt met een afwijking van (RMSE * 2) t.o.v. de lijn

def plot_scatter_regression(country, product):
    # selecteer de input datasets
    dataset_x = normalize(select_plot_foodprices_average(df_foodprices, country, product, 1990, 2020), 'average_price')
    dataset_y = select_plot_migration_movements(df_migration, country, 1990, 2020)

    # de lijsten die worden gegenereerd
    covered_months = []
    output_x = []
    output_y = []
    
    # hovertool
    hover = HoverTool(
        tooltips=[
            ("Normalised foodprice", "$x{1.11}"),
            ("Refugees to Europe", "$y{int}"),
#             ("Month", "{}".format(covered_months[int('$y')])),
        ]
    )
    
    # maak het ontwerp van de grafiek
    f = figure(plot_width=416, plot_height=416, tools=[hover])
    f.title.text_font_size = "8pt"
    f.title.text="Correlation between {} and migration movements".format(product)
    f.xaxis.axis_label = "Normalized foodprices"
    f.yaxis.axis_label = "Migration movements"
    

    # vergelijk de mogelijke maanden en zet ze in een lijst wanneer ze gelijk zijn
    for month_x in dataset_x.year:
        for month_y in dataset_y.Timestamp:    
            if round(month_x, 4) == round(month_y, 4):
                output_x.append(float(dataset_x.loc[dataset_x['year'] == month_x]['Normalized_data']))
                output_y.append(float(dataset_y.loc[dataset_y['Timestamp'] == month_y][0:1]['Value']))
                covered_months.append(month_x)

    # regression line    
    regression_x = np.vstack(output_x)
    regression_x = np.column_stack((regression_x, np.ones(regression_x.shape[0])))
    
    # scale
    x = pd.Series(output_x)
    a, b = np.linalg.lstsq(regression_x, output_y)[0]
    
    # RMSE
    y = a * x + b
    rmse = round(np.sqrt(mean_squared_error(output_y, y)) * 2, 2)

    
    print(country, product, rmse)
    # plot the graph
    f.scatter(output_x, output_y, color='red', legend="RMSE = {}".format(rmse))
    f.line(x, a * x + b, color='blue')
    
    # save
    output_file("{}_{}.html".format(country, product))
    save(f)
#     show(f)
    
plot_scatter_regression('Afghanistan', 'Bread')

Afghanistan Bread 36589.47


c:\users\boble\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [33]:
os.chdir('../plotted comparisons/normalized_regression_plots')
# os.chdir('../regression_plots')

In [53]:
print(os.getcwd())

C:\Users\boble\Documents\GitHub\DataProcessing\plotted comparisons\regression_plots


In [54]:
# Helpfile landen en producten

Africa = ['Central African Republic', 'Djibouti', 'Gambia', 'Guinea', 'Kenya', 'Lesotho', 'Liberia', 'Madagascar', 
                'Malawi', 'Mali', 'Mauritania', 'Mozambique', 'Niger', 'Nigeria', 'Rwanda', 'Senegal', 'South Sudan', 
                'Sudan', 'Uganda', 'Zambia', 'Zimbabwe']
# Kenya, Lesotho veroorzaakte problemen
Middle_East = ['Afghanistan', 'Iraq', 'Iran  (Islamic Republic of)', 'Lebanon', 'State of Palestine', 'Pakistan']
# zie onderste cell voor alle producten

In [56]:
# plot voor alle landen alle producten in een scatter plot met een regression line

for country in Africa:
    for product in products_country(df_foodprices, country):
        try:
            plot_scatter_regression(country, product)
        except:
            print('Error: {}_{}'.format(country, product))

c:\users\boble\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Central African Republic Maize 69.79
Central African Republic Rice 59.64
Central African Republic Wheat flour 23.99
Central African Republic Oil (palm) 89.04
Central African Republic Sorghum 19.9
Central African Republic Millet 58.81
Central African Republic Groundnuts (unshelled) 71.23
Central African Republic Beans (niebe) 53.92
Central African Republic Meat (beef) 94.83
Central African Republic Sesame 23.02
Central African Republic Oil (groundnut) 63.82
Central African Republic Cassava (cossette) 59.08
Djibouti Bread 48.48
Djibouti Wheat flour 45.44
Djibouti Rice (imported) 41.0
Djibouti Sorghum 23.42
Djibouti Beans (white) 48.3
Djibouti Millet 48.31
Djibouti Oil (vegetable) 48.47
Djibouti Sugar 48.47
Djibouti Pasta 47.36
Djibouti Sorghum (white) 52.55
Djibouti Sorghum (red) 52.96
Djibouti Fuel (kerosene) 48.71
Gambia Maize (local) 694.96
Gambia Sorghum 1185.53
Gambia Millet 1444.98
Gambia Groundnuts (unshelled) 1416.4
Gambia Groundnuts (shelled) 1336.21
Gambia Rice (long grain, imp

Zimbabwe Beans 76.93
Zimbabwe Maize 172.71
Zimbabwe Sorghum 81.1
Zimbabwe Millet 71.16
Zimbabwe Wheat 67.73
Zimbabwe Oil (vegetable) 80.78
Zimbabwe Groundnuts (shelled) 83.18
Zimbabwe Cowpeas 81.9
Zimbabwe Maize meal (white, roller) 79.78
Error: Zimbabwe_Oil (vegetable, fortified)
Error: Zimbabwe_Maize meal (white, fortified)
Error: Zimbabwe_Wheat flour (fortified)
Error: Zimbabwe_Sugar (white, fortified)


In [12]:
# plot alle voedselprijzen van een land in een scatterplot

def plot_scatter_regression_all(country):
    # de lijsten die worden gegenereerd
    covered_months = []
    output_x = []
    output_y = []

    # hovertool
    hover = HoverTool(
        tooltips=[
            ("Normalised foodprice", "$x{1.11}"),
            ("Refugees to Europe", "$y{int}"),
#             ("Product", "{}".format(product))
        ]
    )

    # maak het ontwerp van de grafiek
    f = figure(plot_width=600, plot_height=600, tools=[hover])
    f.title.text="Correlation between foodprices and refugees in {}".format(country)
    f.xaxis.axis_label = "Normalized foodprices"
    f.yaxis.axis_label = "Migration movements"

    for product in products_country(df_foodprices, country):
        # selecteer de input datasets
        dataset_x = normalize(select_plot_foodprices_average(df_foodprices, country, product, 1990, 2020), 'average_price')
        dataset_y = select_plot_migration_movements(df_migration, country, 1990, 2020)

        # vergelijk de mogelijke maanden en zet ze in een lijst wanneer ze gelijk zijn
        for month_x in dataset_x.year:
            for month_y in dataset_y.Timestamp:    
                if round(month_x, 4) == round(month_y, 4):
                    output_x.append(float(dataset_x.loc[dataset_x['year'] == month_x]['Normalized_data']))
                    output_y.append(float(dataset_y.loc[dataset_y['Timestamp'] == month_y][0:1]['Value']))
                    covered_months.append(month_x)

    # plot the graph
    f.scatter(output_x, output_y, color='red')

    show(f)
    
plot_scatter_regression_all('Zimbabwe')

In [217]:
# plot genormaliseerde foodprices van een product tegenover genormaliseerde migration movements in een bepaald land

def plot_scatter_regression_normalized(country, product):
    # selecteer de input datasets
    dataset_x = normalize(select_plot_foodprices_average(df_foodprices, country, product, 1990, 2020), 'average_price')
    dataset_y = normalize(select_plot_migration_movements(df_migration, country, 1990, 2020), 'Value')   

    # de lijsten die worden gegenereerd
    covered_months = []
    output_x = []
    output_y = []
    
    # maak het ontwerp van de grafiek
    f = figure(plot_width=600, plot_height=600)
    f.title.text="Correlation between {} and refugees in {}".format(product, country)
    f.xaxis.axis_label = "Normalized foodprices"
    f.yaxis.axis_label = "Normalized migration movements"

    # vergelijk de mogelijke maanden en zet ze in een lijst wanneer ze gelijk zijn
    for month_x in dataset_x.year:
        for month_y in dataset_y.Timestamp:    
            if round(month_x, 4) == round(month_y, 4):
                output_x.append(float(dataset_x.loc[dataset_x['year'] == month_x]['Normalized_data']))
                output_y.append(float(dataset_y.loc[dataset_y['Timestamp'] == month_y][0:1]['Normalized_data']))
                covered_months.append(month_x)

    # regression line    
    regression_x = np.vstack(output_x)
    regression_x = np.column_stack((regression_x, np.ones(regression_x.shape[0])))
    
    # scale
    x = pd.Series(output_x)
    a, b = np.linalg.lstsq(regression_x, output_y)[0]
    
    # plot the graph
    f.circle(output_x, output_y, color='red')
    f.line(x, a * x + b, color='blue')
    
    show(f)
    
plot_scatter_regression_normalized('Gambia', 'Rice (paddy, long grain, local)')

c:\users\boble\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [184]:
# compare_refugees_Gambia_and_price_of_Rice (paddy, long grain, local)

def plot_timeline(country, product, year1, year2):
    # scatter
    x1 = select_plot_foodprices_average(df_foodprices, country, product, year1, year2)['year']
    y1 = normalize(select_plot_foodprices_average(df_foodprices, country, product, year1, year2), 'average_price')['Normalized_data']    
    
    # regression line    
    x2 = np.vstack(select_plot_foodprices_average(df_foodprices, country, product, year1, year2)['year'])
    x2 = np.column_stack((x2, np.ones(x2.shape[0])))
    y2 = normalize(select_plot_foodprices_average(df_foodprices, country, product, year1, year2), 'average_price')['Normalized_data']
    x = select_plot_foodprices_average(df_foodprices, country, product, year1, year2)['year']
    # Now get out m and b values for our best fit line
    a, b = np.linalg.lstsq(x2, y2)[0]

    # set the layout
    f = figure(plot_width=600, plot_height=600)
    
    f.title.text="Price of {} in {} over time".format(product, country)
    f.xaxis.axis_label = "Years"
    f.yaxis.axis_label = "Normalised migration movements"

    # plot the graph
    f.circle(x1, y1, color='red')
    f.line(x, a * x + b, color='blue')

    show(f)
    
plot_timeline('Gambia', 'Rice (paddy, long grain, local)', 2005, 2018)

c:\users\boble\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  
